In [ ]:
!pip install scikit-surprise

import library

In [25]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, average_precision_score
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import numpy as np

load data

In [13]:
file_path = "steam.csv"
data = pd.read_csv(file_path)

file1_path = "steam_desc.csv"
data_desc = pd.read_csv(file1_path)

data preprocessing

In [14]:
cek = data.isnull().sum()
print(cek)
print(data)

cek1 = data_desc.isnull().sum()
print(cek1)
print(data_desc)

appid                0
name                 0
release_date         0
english              0
developer            1
publisher           14
platforms            0
required_age         0
categories           0
genres               0
steamspy_tags        0
achievements         0
positive_ratings     0
negative_ratings     0
average_playtime     0
median_playtime      0
owners               0
price                0
dtype: int64
         appid                        name release_date  english  \
0           10              Counter-Strike   2000-11-01        1   
1           20       Team Fortress Classic   1999-04-01        1   
2           30               Day of Defeat   2003-05-01        1   
3           40          Deathmatch Classic   2001-06-01        1   
4           50   Half-Life: Opposing Force   1999-11-01        1   
...        ...                         ...          ...      ...   
27070  1065230             Room of Pandora   2019-04-24        1   
27071  1065570               

cek data yang hilang dalam steam.csv

In [10]:
missing_publishers = data[data['publisher'].isnull()]
print(missing_publishers[['name', 'publisher']])

missing_dev = data[data['developer'].isnull()]
print(missing_dev[['name', 'developer']])

                                        name publisher
3420                                Borealis       NaN
4511   Glorkian Warrior: The Trials Of Glork       NaN
5229                           Pirate's Life       NaN
7464                      Divergence: Online       NaN
7911                     Melancholy Republic       NaN
9894                            After Dreams       NaN
10011             Interstellar Logistics Inc       NaN
10564                   Ruin of the Reckless       NaN
11123                              Max Stern       NaN
13901                     SuperCluster: Void       NaN
17829                                  Scrap       NaN
22867                      BRANE™: Prototype       NaN
23071                     The Battle Of Ages       NaN
23876                         Holiday Escape       NaN
                     name developer
23071  The Battle Of Ages       NaN


Menggabungkan deskripsi dari data steam_desc.csv ke dalam 1 dataframe yang sama

In [11]:
print(data_desc.columns)
print(data.columns)

Index(['steam_appid', 'short_description'], dtype='object')
Index(['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price'],
      dtype='object')


In [15]:
# Merge dengan data_desc
data = data.merge(data_desc[['steam_appid', 'short_description']],
                  left_on='appid', right_on='steam_appid', how='left')

# Pastikan steam_appid sama dengan appid
data['steam_appid'] = data['appid']

# Hapus steam_appid dari data_desc jika tidak diperlukan
data.drop(columns=['steam_appid'], inplace=True)

# Cek hasilnya
print(data.columns)
print(data[['appid', 'short_description']].head())

Index(['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price',
       'short_description'],
      dtype='object')
   appid                                  short_description
0     10  Play the world's number 1 online action game. ...
1     20  One of the most popular online action games of...
2     30  Enlist in an intense brand of Axis vs. Allied ...
3     40  Enjoy fast-paced multiplayer gaming with Death...
4     50  Return to the Black Mesa Research Facility as ...


feature engineering

In [17]:
data = data[['name', 'positive_ratings', 'negative_ratings', 'average_playtime', 'genres', 'categories', 'price', 'owners', 'platforms',
             'steamspy_tags', 'short_description']]

print(data.isnull())

# filter data untuk memastikan game yang diseleksi
data = data[(data['positive_ratings'] + data['negative_ratings'] >= 100) & (data['average_playtime'] > 0)]

# buat variable baru untuk formula satisfaction score
data['satisfaction_score'] = (data['positive_ratings'] / (data['positive_ratings'] + data['negative_ratings'])) * data['average_playtime']


        name  positive_ratings  negative_ratings  average_playtime  genres  \
0      False             False             False             False   False   
1      False             False             False             False   False   
2      False             False             False             False   False   
3      False             False             False             False   False   
4      False             False             False             False   False   
...      ...               ...               ...               ...     ...   
27070  False             False             False             False   False   
27071  False             False             False             False   False   
27072  False             False             False             False   False   
27073  False             False             False             False   False   
27074  False             False             False             False   False   

       categories  price  owners  platforms  steamspy_tags  sho

Model 1 - TF-IDF + Cosine Similarity

In [ ]:
# prompt: buatkan code Content-Based Filtering (CBF)  lagi dengan fitur yang berbeda

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import numpy as np

!pip install scikit-surprise

# Load data
file_path = "steam.csv"
data = pd.read_csv(file_path)

# Feature Engineering (enhanced)
data = data[['name', 'positive_ratings', 'negative_ratings', 'average_playtime', 'genres', 'categories', 'owners']]

# Data Cleaning
data = data.dropna()  # Remove rows with missing values
data['owners'] = data['owners'].astype(str).str.replace(',', '').str.replace('-', ' ').str.split().str[0].fillna(0).astype(int)
data = data[(data['positive_ratings'] + data['negative_ratings'] >= 100) & (data['average_playtime'] > 0) & (data['owners'] > 0)]

# Calculate a more nuanced satisfaction score
data['satisfaction_score'] = ((data['positive_ratings'] / (data['positive_ratings'] + data['negative_ratings'])) * 0.6 +
                             (data['average_playtime'] / data['average_playtime'].max()) * 0.4) * np.log1p(data['owners'])

# Content-Based Filtering (CBF) with enhanced features
def content_based_filtering(data, game_name, top_n=10):
    data['text_features'] = data['name'] + ' ' + data['genres'] + ' ' + data['categories']

    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(data['text_features'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    indices = pd.Series(data.index, index=data['name']).drop_duplicates()

    if game_name not in indices:
        return f"Game '{game_name}' not found in dataset."

    idx = indices[game_name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n + 1]  # Exclude the game itself

    game_indices = [i[0] for i in sim_scores]
    return data.iloc[game_indices][['name', 'genres', 'categories', 'owners']]


# Example Usage
example_game_name = "Team Fortress Classic"  # Example game name

# Content-Based Filtering
print("Content-Based Filtering Recommendations:")
cbf_results = content_based_filtering(data, example_game_name)
cbf_results

Content-Based Filtering Recommendations:


,name,genres,categories,owners
3,Deathmatch Classic,Action,Multi-player;Online Multi-Player;Local Multi-P...,5000000
19,Team Fortress 2,Action;Free to Play,Multi-player;Cross-Platform Multiplayer;Steam ...,20000000
0,Counter-Strike,Action,Multi-player;Online Multi-Player;Local Multi-P...,10000000
5,Ricochet,Action,Multi-player;Online Multi-Player;Valve Anti-Ch...,5000000
6,Half-Life,Action,Single-player;Multi-player;Online Multi-Player...,5000000
9248,Call of Duty®: WWII,Action,Single-player;Multi-player;Online Multi-Player...,500000
3997,Out of Reach,Action;Adventure;Indie;Massively Multiplayer;R...,Single-player;Multi-player;Online Multi-Player...,50000
3943,TOXIKK™,Action;Free to Play;Indie,Single-player;Multi-player;Online Multi-Player...,500000
6494,Squad,Action;Indie;Strategy;Early Access,Multi-player;Online Multi-Player;Steam Trading...,500000
1506,DayZ,Action;Adventure;Massively Multiplayer,Multi-player;Online Multi-Player;Steam Worksho...,2000000


# **experiment 1**

In [ ]:
# prompt: dari code line [89] diatas lakukan experiment kepada code  bagian Content-Based Filtering  dengan ubah fitur yang digunakan selain name, genres, categories

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import numpy as np

# Load data (assuming 'steam.csv' is in the same directory)
file_path = "steam.csv"
data = pd.read_csv(file_path)

# Feature Engineering (enhanced)
data = data[['name', 'positive_ratings', 'negative_ratings', 'average_playtime', 'genres', 'categories', 'owners', 'platforms']]

# Data Cleaning (handling missing values and data type conversion)
data = data.dropna()  # Remove rows with missing values
data['owners'] = data['owners'].astype(str).str.replace(',', '').str.replace('-', ' ').str.split().str[0].fillna(0).astype(int)
data = data[(data['positive_ratings'] + data['negative_ratings'] >= 100) & (data['average_playtime'] > 0) & (data['owners'] > 0)]

# Calculate a more nuanced satisfaction score
data['satisfaction_score'] = ((data['positive_ratings'] / (data['positive_ratings'] + data['negative_ratings'])) * 0.6 +
                             (data['average_playtime'] / data['average_playtime'].max()) * 0.4) * np.log1p(data['owners'])

# Content-Based Filtering (CBF) with MODIFIED features
def content_based_filtering(data, game_name, top_n=10):
    # MODIFIED: Using 'genres', 'categories', and 'platforms'
    data['text_features'] = data['genres'] + ' ' + data['categories'] + ' ' + data['platforms']  # Combine relevant features

    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(data['text_features'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    indices = pd.Series(data.index, index=data['name']).drop_duplicates()

    if game_name not in indices:
        return f"Game '{game_name}' not found in dataset."

    idx = indices[game_name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n + 1]  # Exclude the game itself

    game_indices = [i[0] for i in sim_scores]
    # MODIFIED: Returning additional relevant features
    return data.iloc[game_indices][['name', 'genres', 'categories', 'platforms', 'owners']]


# Example Usage
example_game_name = "Ricochet"  # Example game name

# Content-Based Filtering
print("Content-Based Filtering Recommendations:")
cbf_results = content_based_filtering(data, example_game_name)
cbf_results

<ipython-input-6-61dd9a8a89bc>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['satisfaction_score'] = ((data['positive_ratings'] / (data['positive_ratings'] + data['negative_ratings'])) * 0.6 +
<ipython-input-6-61dd9a8a89bc>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text_features'] = data['genres'] + ' ' + data['categories'] + ' ' + data['platforms']  # Combine relevant features


Content-Based Filtering Recommendations:


,name,genres,categories,platforms,owners
6,Half-Life,Action,Single-player;Multi-player;Online Multi-Player...,windows;mac;linux,5000000
2,Day of Defeat,Action,Multi-player;Valve Anti-Cheat enabled,windows;mac;linux,5000000
15,Half-Life Deathmatch: Source,Action,Multi-player;Valve Anti-Cheat enabled,windows;mac;linux,5000000
4,Half-Life: Opposing Force,Action,Single-player;Multi-player;Valve Anti-Cheat en...,windows;mac;linux,5000000
7,Counter-Strike: Condition Zero,Action,Single-player;Multi-player;Valve Anti-Cheat en...,windows;mac;linux,10000000
0,Counter-Strike,Action,Multi-player;Online Multi-Player;Local Multi-P...,windows;mac;linux,10000000
1,Team Fortress Classic,Action,Multi-player;Online Multi-Player;Local Multi-P...,windows;mac;linux,5000000
3,Deathmatch Classic,Action,Multi-player;Online Multi-Player;Local Multi-P...,windows;mac;linux,5000000
27,Red Orchestra: Ostfront 41-45,Action,Multi-player;Steam Achievements;Valve Anti-Che...,windows;mac;linux,500000
5281,The Mean Greens - Plastic Warfare,Action;Indie,Multi-player;Online Multi-Player;Steam Achieve...,windows;linux,200000


# **experiment 2**

In [ ]:
# prompt: dari code line [93] diatas lakukan experiment kepada code  bagian Content-Based Filtering  dengan ubah fitur yang digunakan selain fitur ini : genres, categories, platforms, price

# Content-Based Filtering (CBF) with MODIFIED features
def content_based_filtering(data, game_name, top_n=10):
    # MODIFIED: Using 'owners', 'platforms', and 'average_playtime'
    # Convert average_playtime to string for concatenation
    data['average_playtime'] = data['average_playtime'].astype(str)
    data['text_features'] = data['owners'].astype(str) + ' ' + data['platforms'] + ' ' + data['average_playtime']  # Combine relevant features

    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(data['text_features'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    indices = pd.Series(data.index, index=data['name']).drop_duplicates()

    if game_name not in indices:
        return f"Game '{game_name}' not found in dataset."

    idx = indices[game_name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n + 1]  # Exclude the game itself

    game_indices = [i[0] for i in sim_scores]
    # MODIFIED: Returning additional relevant features
    return data.iloc[game_indices][['name', 'owners', 'platforms', 'average_playtime']]


# Example Usage
example_game_name = "Day of Defeat"  # Example game name

# Content-Based Filtering
print("Content-Based Filtering Recommendations:")
cbf_results = content_based_filtering(data, example_game_name)
cbf_results

Content-Based Filtering Recommendations:


,name,owners,platforms,average_playtime
2100,Project Nimbus,20000,windows;mac,187
4425,Retention,200000,windows,187
10207,Ashes of the Singularity: Escalation,200000,windows,187
3271,The Blue Flamingo,100000,windows,187
10047,Batman™: Arkham VR,50000,windows,187
11598,"GO AWAY, THERE'S KUMIS OVER THERE!",50000,windows,187
13887,Zup! Zero,50000,windows,187
17002,Generation Zero,50000,windows,187
19913,Otaku's Fantasy 2,50000,windows,187
16868,Selenophobia,20000,windows,187


Model 2. Count Vectorizer + Cosine Similarity

Experiment 1 : Genre, kategori, dan deskripsi

In [32]:
data['combined_features'] = data['genres'] + ' ' + data['categories'] + ' ' + data['short_description']

vectorizer = CountVectorizer(stop_words='english')
count_matrix = vectorizer.fit_transform(data['combined_features'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# menerima input untuk menrekomendasi game
def get_game_recommendations(game_name, cosine_sim=cosine_sim):
    try:
        idx = data.index[data['name'] == game_name].tolist()[0]
    except IndexError:
        return f"Game '{game_name}' tidak ditemukan, pastikan nama game sudah benar."

    sim_scores = list(enumerate(cosine_sim[idx]))

    # Mengurutkan game berdasarkan skor kesamaan (dari yang paling mirip)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Mengambil 5 game teratas selain game itu sendiri
    sim_scores = sim_scores[1:6]

    # Mendapatkan indeks game yang relevan
    game_indices = [i[0] for i in sim_scores]

    # Mengembalikan nama game yang direkomendasikan
    return data['name'].iloc[game_indices]

recommended_games = get_game_recommendations('Counter-Strike')
print(recommended_games)

1                      Team Fortress Classic
5                                   Ricochet
13134                  MadOut2 BigCityOnline
17449    東方憑依華　～ Antinomy of Common Flowers.
1576                              Sven Co-op
Name: name, dtype: object


Experiment 2 : Genre, kategori, steamspy tag, dan deskripsi

In [33]:
data['combined_features'] = data['genres'] + ' ' + data['categories'] + ' ' + data['steamspy_tags'] + ' ' + data['short_description']

vectorizer = CountVectorizer(stop_words='english')
count_matrix = vectorizer.fit_transform(data['combined_features'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# menerima input untuk menrekomendasi game
def get_game_recommendations(game_name, cosine_sim=cosine_sim):
    try:
        idx = data.index[data['name'] == game_name].tolist()[0]
    except IndexError:
        return f"Game '{game_name}' tidak ditemukan, pastikan nama game sudah benar."

    sim_scores = list(enumerate(cosine_sim[idx]))

    # Mengurutkan game berdasarkan skor kesamaan (dari yang paling mirip)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Mengambil 5 game teratas selain game itu sendiri
    sim_scores = sim_scores[1:6]

    # Mendapatkan indeks game yang relevan
    game_indices = [i[0] for i in sim_scores]

    # Mengembalikan nama game yang direkomendasikan
    return data['name'].iloc[game_indices]

recommended_games = get_game_recommendations('Counter-Strike')
print(recommended_games)

1                  Team Fortress Classic
5                               Ricochet
6                              Half-Life
2991     Call of Duty®: Infinite Warfare
16331                          Battlegun
Name: name, dtype: object


Experiment 3 : Genre dan deskripsi

In [37]:
data['combined_features'] = data['genres'] + ' ' + data['short_description']

vectorizer = CountVectorizer(stop_words='english')
count_matrix = vectorizer.fit_transform(data['combined_features'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# menerima input untuk menrekomendasi game
def get_game_recommendations(game_name, cosine_sim=cosine_sim):
    try:
        idx = data.index[data['name'] == game_name].tolist()[0]
    except IndexError:
        return f"Game '{game_name}' tidak ditemukan, pastikan nama game sudah benar."

    sim_scores = list(enumerate(cosine_sim[idx]))

    # Mengurutkan game berdasarkan skor kesamaan (dari yang paling mirip)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Mengambil 5 game teratas selain game itu sendiri
    sim_scores = sim_scores[1:6]

    # Mendapatkan indeks game yang relevan
    game_indices = [i[0] for i in sim_scores]

    # Mengembalikan nama game yang direkomendasikan
    return data['name'].iloc[game_indices]

recommended_games = get_game_recommendations('Counter-Strike')
print(recommended_games)

11091           Smashbox Arena
1        Team Fortress Classic
8859               Hero Battle
13134    MadOut2 BigCityOnline
22015       Touhou Luna Nights
Name: name, dtype: object


Experiment 4 : Genre dan kategori

In [38]:
data['combined_features'] = data['genres'] + ' ' + data['categories']

vectorizer = CountVectorizer(stop_words='english')
count_matrix = vectorizer.fit_transform(data['combined_features'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# menerima input untuk menrekomendasi game
def get_game_recommendations(game_name, cosine_sim=cosine_sim):
    try:
        idx = data.index[data['name'] == game_name].tolist()[0]
    except IndexError:
        return f"Game '{game_name}' tidak ditemukan, pastikan nama game sudah benar."

    sim_scores = list(enumerate(cosine_sim[idx]))

    # Mengurutkan game berdasarkan skor kesamaan (dari yang paling mirip)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Mengambil 5 game teratas selain game itu sendiri
    sim_scores = sim_scores[1:6]

    # Mendapatkan indeks game yang relevan
    game_indices = [i[0] for i in sim_scores]

    # Mengembalikan nama game yang direkomendasikan
    return data['name'].iloc[game_indices]

recommended_games = get_game_recommendations('Counter-Strike')
print(recommended_games)

1               Team Fortress Classic
3                  Deathmatch Classic
5                            Ricochet
6                           Half-Life
144    Cossacks II: Battle for Europe
Name: name, dtype: object
